# Robôzinho para coletar informações de ETFs na internet

## Passo 1 - importando e instalando bibliotecas

In [129]:
#!pip install selenium
#!pip install webdriver-manager

In [130]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd
import pdb

## Passo 2 - Abrindo o navegador

In [131]:
driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()))
driver.get('https://www.etf.com/etfanalytics/etf-finder')

## Passo 3 - Selecionando botão para aumentar visualização da tabela

In [132]:
time.sleep(5)

botao_100= driver.find_element('xpath', '/html/body/div[2]/div/div/main/div[1]/section/div[2]/div[3]/div/article/div/div[2]/div/div/div/div/div/div[3]/div[2]/div/div[1]/div/div[2]/div[1]/div/div[5]/button/span')

#botao_100.click()

# Outra forma para clicar em um objeto do HTML
driver.execute_script('arguments[0].click();', botao_100) 

## Passo 4 - Encontrando o número total de páginas da tabela

In [136]:
numero_paginas = driver.find_element('xpath', '/html/body/div[2]/div/div/main/div[1]/section/div[2]/div[3]/div/article/div/div[2]/div/div/div/div/div/div[3]/div[2]/div/div[1]/div/div[2]/div[2]/ul/li[8]/a')
numero_paginas = int(numero_paginas.text)
numero_paginas

32

## Passo 5 - Lendo tabela de dados

In [139]:
elemento = driver.find_element('xpath', '//*[@id="panel:r1:0"]/div/div[1]')

In [140]:
botao_avancar_pagina = driver.find_element('xpath', '/html/body/div[2]/div/div/main/div[1]/section/div[2]/div[3]/div/article/div/div[2]/div/div/div/div/div/div[3]/div[2]/div/div[1]/div/div[2]/div[2]/ul/li[9]/a[1]')
lista_tabelas = []
for pagina in range(numero_paginas):
    #Exportando a tabela em HTML
    html_tabela = elemento.get_attribute('outerHTML')
    
    #Utilizando tabela exportada com pandas e salvando em uma lista com todas as outras tabelas
    tabela = pd.read_html(str(html_tabela))[0]
    lista_tabelas.append(tabela)
    
    if pagina == 0:
        driver.execute_script('arguments[0].click();', botao_avancar_pagina)
        
    elif pagina < numero_paginas-1:
        try:    
            if botao_avancar_pagina.text != "Next":
                botao_avancar_pagina = driver.find_element('xpath', '//*[@id="panel:r0:0"]/div/div[2]/div[2]/ul/li[11]/a[1]')
                if botao_avancar_pagina.text != "Next":
                    botao_avancar_pagina = driver.find_element('xpath', '//*[@id="panel:r0:0"]/div/div[2]/div[2]/ul/li[10]/a[1]')
                else:
                    botao_avancar_pagina = driver.find_element('xpath', '//*[@id="panel:r0:0"]/div/div[2]/div[2]/ul/li[9]/a[1]')
 
        except:
            botao_avancar_pagina = driver.find_element('xpath', '//*[@id="panel:r0:0"]/div/div[2]/div[2]/ul/li[10]/a[1]')
            if botao_avancar_pagina.text != "Next":
                botao_avancar_pagina = driver.find_element('xpath', '//*[@id="panel:r0:0"]/div/div[2]/div[2]/ul/li[9]/a[1]')
            pass

        finally:
            driver.execute_script('arguments[0].click();', botao_avancar_pagina)

tabela_etfs = pd.concat(lista_tabelas)
tabela_etfs.to_excel("tabela_etfs.xlsx", index = False)

[WDM] - Downloading: 100%|████████████████████████████████████████████████████████| 6.80M/6.80M [04:23<00:00, 27.1kB/s]


In [141]:
tabela_etfs

,Ticker,Fund Name,Segment,1 YR Return,Expense Ratio,AUM
0,SPY,SPDR S&P 500 ETF Trust,Equity: U.S. - Large Cap,-5.14%,0.09%,$373.61B
1,IVV,iShares Core S&P 500 ETF,Equity: U.S. - Large Cap,-5.14%,0.03%,$304.89B
2,VOO,Vanguard S&P 500 ETF,Equity: U.S. - Large Cap,-5.18%,0.03%,$285.87B
3,VTI,Vanguard Total Stock Market ETF,Equity: U.S. - Total Market,-6.35%,0.03%,$280.56B
4,QQQ,Invesco QQQ Trust,Equity: U.S. - Large Cap,-7.14%,0.20%,$169.61B
...,...,...,...,...,...,...
22,CDEI,"Calvert US Large-Cap Diversity, Equity and Inc...",Equity: U.S. - Large Cap,--,0.14%,--
23,HAPS,Harbor Corporate Culture Small Cap ETF,Equity: U.S. - Small Cap,--,0.64%,--
24,CVSE,Calvert US Select Equity ETF,Equity: U.S. - Large Cap,--,0.29%,--
25,SROI,Calamos Antetokounmpo Global Sustainable Equit...,Equity: Global - Total Market,--,0.95%,--


## Passo 6 - Coletando dados de performance

In [142]:
botao_retorna_primeira_pagina = driver.find_element('xpath', '//*[@id="panel:r1:0"]/div/div[2]/div[2]/ul/li[1]/a[1]')
driver.execute_script('arguments[0].click();', botao_retorna_primeira_pagina)

botao_performance = driver.find_element('xpath', '//*[@id="tab:r1:1"]')
driver.execute_script('arguments[0].click();',botao_performance)

In [143]:
time.sleep(2)
botao_100 = driver.find_element('xpath', '//*[@id="panel:r1:1"]/div/div[2]/div[1]/div/div[5]/button')
driver.execute_script('arguments[0].click();',botao_100)

botao_avancar_pagina = driver.find_element('xpath', '//*[@id="panel:r1:1"]/div/div[2]/div[2]/ul/li[9]/a[1]')

In [144]:
lista_tabelas = []
elemento = driver.find_element('xpath', '//*[@id="panel:r1:1"]/div/div[1]')
for pagina in range(numero_paginas):
    #Exportando a tabela em HTML
    html_tabela = elemento.get_attribute('outerHTML')
    
    #Utilizando tabela exportada com pandas e salvando em uma lista com todas as outras tabelas
    tabela = pd.read_html(str(html_tabela))[0]
    lista_tabelas.append(tabela)
    
    if pagina == 0:
        driver.execute_script('arguments[0].click();', botao_avancar_pagina)
        
    elif pagina < numero_paginas-1:
        try:    
            if botao_avancar_pagina.text != "Next":
                botao_avancar_pagina = driver.find_element('xpath', '//*[@id="panel:r0:0"]/div/div[2]/div[2]/ul/li[11]/a[1]')
                if botao_avancar_pagina.text != "Next":
                    botao_avancar_pagina = driver.find_element('xpath', '//*[@id="panel:r0:0"]/div/div[2]/div[2]/ul/li[10]/a[1]')
                else:
                    botao_avancar_pagina = driver.find_element('xpath', '//*[@id="panel:r0:0"]/div/div[2]/div[2]/ul/li[9]/a[1]')
 
        except:
            botao_avancar_pagina = driver.find_element('xpath', '//*[@id="panel:r0:0"]/div/div[2]/div[2]/ul/li[10]/a[1]')
            if botao_avancar_pagina.text != "Next":
                botao_avancar_pagina = driver.find_element('xpath', '//*[@id="panel:r0:0"]/div/div[2]/div[2]/ul/li[9]/a[1]')
            pass

        finally:
            driver.execute_script('arguments[0].click();', botao_avancar_pagina)

tabela_performance_etfs = pd.concat(lista_tabelas)
tabela_performance_etfs

,Ticker,Fund Name,1 MO,3 MO,YTD,1 YR,3 YR,5 YR,10 YR,As Of Date
0,SPY,SPDR S&P 500 ETF Trust,7.71%,4.16%,8.53%,-5.14%,16.22%,11.19%,12.11%,04/12/2023
1,IVV,iShares Core S&P 500 ETF,7.73%,4.15%,8.54%,-5.14%,16.29%,11.22%,12.16%,04/12/2023
2,VOO,Vanguard S&P 500 ETF,7.69%,4.12%,8.53%,-5.18%,16.28%,11.21%,12.16%,04/12/2023
3,VTI,Vanguard Total Stock Market ETF,7.28%,3.22%,8.00%,-6.35%,16.04%,10.38%,11.66%,04/12/2023
4,QQQ,Invesco QQQ Trust,9.97%,13.77%,20.05%,-7.14%,16.98%,15.44%,17.47%,04/12/2023
...,...,...,...,...,...,...,...,...,...,...
22,CDEI,"Calvert US Large-Cap Diversity, Equity and Inc...",7.67%,--,--,--,--,--,--,04/12/2023
23,HAPS,Harbor Corporate Culture Small Cap ETF,--,--,--,--,--,--,--,04/12/2023
24,CVSE,Calvert US Select Equity ETF,6.79%,--,--,--,--,--,--,04/12/2023
25,SROI,Calamos Antetokounmpo Global Sustainable Equit...,7.02%,--,--,--,--,--,--,04/12/2023


In [145]:
tabela_performance_etfs.to_excel("tabela_performance_etfs.xlsx", index = False)

In [146]:
#Fechando o navegador
driver.quit()

## Passo 7 - Construindo tabela final

In [147]:
display(tabela_performance_etfs)

,Ticker,Fund Name,1 MO,3 MO,YTD,1 YR,3 YR,5 YR,10 YR,As Of Date
0,SPY,SPDR S&P 500 ETF Trust,7.71%,4.16%,8.53%,-5.14%,16.22%,11.19%,12.11%,04/12/2023
1,IVV,iShares Core S&P 500 ETF,7.73%,4.15%,8.54%,-5.14%,16.29%,11.22%,12.16%,04/12/2023
2,VOO,Vanguard S&P 500 ETF,7.69%,4.12%,8.53%,-5.18%,16.28%,11.21%,12.16%,04/12/2023
3,VTI,Vanguard Total Stock Market ETF,7.28%,3.22%,8.00%,-6.35%,16.04%,10.38%,11.66%,04/12/2023
4,QQQ,Invesco QQQ Trust,9.97%,13.77%,20.05%,-7.14%,16.98%,15.44%,17.47%,04/12/2023
...,...,...,...,...,...,...,...,...,...,...
22,CDEI,"Calvert US Large-Cap Diversity, Equity and Inc...",7.67%,--,--,--,--,--,--,04/12/2023
23,HAPS,Harbor Corporate Culture Small Cap ETF,--,--,--,--,--,--,--,04/12/2023
24,CVSE,Calvert US Select Equity ETF,6.79%,--,--,--,--,--,--,04/12/2023
25,SROI,Calamos Antetokounmpo Global Sustainable Equit...,7.02%,--,--,--,--,--,--,04/12/2023


In [159]:
tabela_rentabilidade_etfs = tabela_performance_etfs.set_index("Ticker")
tabela_rentabilidade_etfs = tabela_rentabilidade_etfs[['1 YR', '3 YR', '5 YR']],
tabela_cadastro_etfs = tabela_etfs.set_index("Ticker")

In [160]:
tabela_dados_final = tabela_cadastro_etfs.join(tabela_rentabilidade_etfs, how = 'inner')
tabela_dados_final

,Fund Name,Segment,1 YR Return,Expense Ratio,AUM,1 YR,3 YR,5 YR
Ticker,,,,,,,,
SPY,SPDR S&P 500 ETF Trust,Equity: U.S. - Large Cap,-5.14%,0.09%,$373.61B,-5.14%,16.22%,11.19%
IVV,iShares Core S&P 500 ETF,Equity: U.S. - Large Cap,-5.14%,0.03%,$304.89B,-5.14%,16.29%,11.22%
VOO,Vanguard S&P 500 ETF,Equity: U.S. - Large Cap,-5.18%,0.03%,$285.87B,-5.18%,16.28%,11.21%
VTI,Vanguard Total Stock Market ETF,Equity: U.S. - Total Market,-6.35%,0.03%,$280.56B,-6.35%,16.04%,10.38%
QQQ,Invesco QQQ Trust,Equity: U.S. - Large Cap,-7.14%,0.20%,$169.61B,-7.14%,16.98%,15.44%
...,...,...,...,...,...,...,...,...
CDEI,"Calvert US Large-Cap Diversity, Equity and Inc...",Equity: U.S. - Large Cap,--,0.14%,--,--,--,--
HAPS,Harbor Corporate Culture Small Cap ETF,Equity: U.S. - Small Cap,--,0.64%,--,--,--,--
CVSE,Calvert US Select Equity ETF,Equity: U.S. - Large Cap,--,0.29%,--,--,--,--


In [161]:
tabela_dados_final.to_excel("tabela_dados_finais.xlsx")